In [1]:
from pyscf import scf,gto
import pyscf
import numpy as np
import inspect
from FcMole import FcM
from functools import reduce
import matplotlib.pyplot as plt
from pyscf.scf import cphf
from pyscf.grad import rhf as grhf  #### very important
from pyscf.hessian import rhf as hrhf # without those two mf.Gradients() and mf.Hessian() don't work
def DeltaV(mol,dL):
    mol.set_rinv_orig_(mol.atom_coords()[0])
    dV=mol.intor('int1e_rinv')*dL[0]
    mol.set_rinv_orig_(mol.atom_coords()[1])
    dV+=mol.intor('int1e_rinv')*dL[1]
    return -dV

In [2]:
mol=gto.M(atom="H 0 0 0; H 0 0 1.4",unit="Bohr",basis="STO-3G")
mf=scf.RHF(mol)
mf.scf(dm0=mf.init_guess_by_atom())
P=mf.make_rdm1()
hcore=mf.get_hcore()
h=mf.Hessian()
h.kernel()
pass

converged SCF energy = -1.11671432506255


/home/giorgiod/miniconda3/lib/python3.7/site-packages/pyscf/lib/chkfile.py:71: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  return val.value


In [3]:
"""def make_h1( mo_coeff, mo_occ, chkfile=None, atmlst=None, verbose=None):
  #  mol = hessobj.mol
    if atmlst is None:
        atmlst = range(mol.natm)
    nao, nmo = mo_coeff.shape
    mocc = mo_coeff[:,mo_occ>0]
    dm0 = numpy.dot(mocc, mocc.T) * 2
    hcore_deriv = hessobj.base.nuc_grad_method().hcore_generator(mol)   #dH/dx need to adj for new charges
    aoslices = mol.aoslice_by_atom()
    h1ao = [None] * mol.natm
    for i0, ia in enumerate(atmlst):
        shl0, shl1, p0, p1 = aoslices[ia]
        shls_slice = (shl0, shl1) + (0, mol.nbas)*3
        vj1, vj2, vk1, vk2 = _get_jk(mol, 'int2e_ip1', 3, 's2kl',
                                     ['ji->s2kl', -dm0[:,p0:p1],  # vj1
                                      'lk->s1ij', -dm0         ,  # vj2
                                      'li->s1kj', -dm0[:,p0:p1],  # vk1
                                      'jk->s1il', -dm0         ], # vk2
                                     shls_slice=shls_slice)
        vhf = vj1 - vk1*.5
        vhf[:,p0:p1] += vj2 - vk2*.5
        h1 = vhf + vhf.transpose(0,2,1)
        h1 += hcore_deriv(ia)
"""
pass
"""def hcore_generator(mf, mol=None):    # on gradient module 
    aoslices = mol.aoslice_by_atom()
    h1 = mf.get_hcore(mol)   # gradients method : -\nabla_r (T+V_en)    is different between mol and fmol
    def hcore_deriv(atm_id):
        shl0, shl1, p0, p1 = aoslices[atm_id]
        with mol.with_rinv_at_nucleus(atm_id):
            vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla|1/r|>    this is equal 
            vrinv *= -mol.atom_charge(atm_id)        # atom charges should be updated 
        vrinv[:,p0:p1] += h1[:,p0:p1]
        return vrinv + vrinv.transpose(0,2,1)
    return hcore_deriv
"""
pass
"""def get_hcore(mol):    # on gradient module 
    '''Part of the nuclear gradients of core Hamiltonian'''
    h = mol.intor('int1e_ipkin', comp=3)
    h+= mol.intor('int1e_ipnuc', comp=3)
    return -h"""
pass

In [8]:
"""def hcore_generator(mf, mol=None):    # on gradient module 
    aoslices = mol.aoslice_by_atom()
    h1 = mf.get_hcore(mol)   # gradients method : -\nabla_r (T+V_en)    is different between mol and fmol
    def hcore_deriv(atm_id):
        shl0, shl1, p0, p1 = aoslices[atm_id]
        with mol.with_rinv_at_nucleus(atm_id):
            vrinv = mol.intor('int1e_iprinv', comp=3) # <\nabla_R 1/|R-r|>    this is equal 
            vrinv *= -mol.atom_charge(atm_id)        # atom charges should be updated 
        vrinv[:,p0:p1] += h1[:,p0:p1]
        return vrinv + vrinv.transpose(0,2,1)
    return hcore_deriv
"""
pass

Basically there are three components: the first is when you move the A.O. the nuclear electron attraction integrals change, as well as the kinetic operator (part two). The third part is when you move the nucleus all the electron nuclear integrals,except those amon A.O. placed on the atoms itself are affected    

In [36]:
mol.aoslice_by_atom()

array([[0, 1, 0, 1],
       [1, 2, 1, 2]])

In [37]:
mol.intor('int1e_ipnuc', comp=3)

array([[[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.        ,  0.        ],
        [ 0.        ,  0.        ]],

       [[ 0.17162944,  0.72602207],
        [-0.72602207, -0.17162944]]])

In [38]:
mol.intor('int1e_ipkin', comp=3)

array([[[ 0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00]],

       [[ 0.00000000e+00, -3.15282344e-01],
        [ 3.15282344e-01, -5.29599004e-17]]])

In [39]:
atm_id=0
with mol.with_rinv_at_nucleus(atm_id):
    print(mol.intor('int1e_iprinv', comp=3))

[[[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.          0.        ]
  [ 0.          0.        ]]

 [[ 0.         -0.25779227]
  [ 0.4682298   0.17162944]]]


In [40]:
a=mol.intor('int1e_iprinv', comp=3)[2]
a

array([[ 0.        , -0.25779227],
       [ 0.4682298 ,  0.17162944]])

In [41]:
-mol.intor('int1e_ipnuc', comp=3)[2][:,0:1]

array([[-0.17162944],
       [ 0.72602207]])

In [42]:
a[:,0:1]+=mol.intor('int1e_ipnuc', comp=3)[2][:,0:1]
a

array([[ 0.17162944, -0.25779227],
       [-0.25779227,  0.17162944]])

In [43]:
a+=a.T
a

array([[ 0.34325889, -0.51558454],
       [-0.51558454,  0.34325889]])